# Modeling and Analysis
## 0. Load datasets

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pyarrow import parquet
from scipy import stats
from statsmodels.regression.mixed_linear_model import MixedLM
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.graphics.gofplots import qqplot
from sklearn.mixture import GaussianMixture
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_absolute_error

In [ ]:
df = pd.read_parquet('/kaggle/input/merged-new/8_joined_8day_no.parquet')

## 1. Pre-Processing

In [ ]:
df = df[df['Name']!='Senja-Grasmyrskogen']
df.loc[df['T_air_night'] > 30, 'T_air_night'] = np.nan

In [ ]:
# Merge day and night observations
df_day = df.copy()
df_day['Modis'] = df_day['LST_Day_1km']
df_day['Situ'] = df_day['T_air_day']
df_day['QC'] = df_day['QC_Day']
df_day['Day_Night'] = 1  # 1 for Day
df_day = df_day.drop(columns=['LST_Day_1km', 'LST_Night_1km', 'T_air_day', 'T_air_night', 'QC_Day', 'QC_Night'])

df_night = df.copy()
df_night['Modis'] = df_night['LST_Night_1km']
df_night['Situ'] = df_night['T_air_night']
df_night['QC'] = df_night['QC_Night']
df_night['Day_Night'] = 0  # 0 for Night
df_night = df_night.drop(columns=['LST_Day_1km', 'LST_Night_1km', 'T_air_day', 'T_air_night', 'QC_Day', 'QC_Night'])

df_combined = pd.concat([df_day, df_night], ignore_index=True)

In [ ]:
df_combined = df_combined.dropna(subset=['Situ','Modis','QC'])

### Encode data and compute statistics

In [ ]:
# Cycling of date
df_combined['Date'] = pd.to_datetime(df_combined['Date'])
df_combined['Month'] = df_combined['Date'].dt.month
df_combined['Year'] = df_combined['Date'].dt.year
df_combined['day_of_year'] = df_combined['Date'].dt.dayofyear
df_combined = df_combined[df_combined['Year'] > 2011]

# Non-linear terms
df_combined['nonlinear1'] = 1 / (1 + np.abs(df_combined['Situ']))
df_combined['nonlinear2'] = 1 / (1 + np.abs(df_combined['Modis']))

# Sine and cosine transformations
df_combined['day_of_year_sin'] = np.sin(2 * np.pi * df_combined['day_of_year'] / 365)
df_combined['day_of_year_cos'] = np.cos(2 * np.pi * df_combined['day_of_year'] / 365)

# Compute monthly statistics for each location
df_combined['monthly_avg'] = df_combined.groupby(['Latitude', 'Longitude', 'Year', 'Month', 'Day_Night'])['Modis'].transform('mean')
df_combined['monthly_med'] = df_combined.groupby(['Latitude', 'Longitude', 'Year', 'Month', 'Day_Night'])['Modis'].transform('median')
df_combined['temporal_dev3'] = df_combined['Modis'] - df_combined['monthly_med']

In [ ]:
# QC Encoding
custom_dict = {
    0: {'1': 0, '2': 0, '3': 0},
    2: {'1': 0, '2': 1, '3': 0},
    3: {'1': 0, '2': 0, '3': 1},
    5: {'1': 0, '2': 0, '3': 0},
    8: {'1': 0, '2': 1, '3': 0},
    17: {'1': 1, '2': 0, '3': 0},
    25: {'1': 1, '2': 1, '3': 0},
    65: {'1': 0, '2': 1, '3': 0},
    73: {'1': 0, '2': 1, '3': 0},
    81: {'1': 1, '2': 1, '3': 0},
    89: {'1': 1, '2': 1, '3': 0},
    129: {'1': 0, '2': 1, '3': 0},
    137: {'1': 0, '2': 1, '3': 0},
    145: {'1': 1, '2': 1, '3': 0},
    197: {'1': 0, '2': 1, '3': 0}
}

In [ ]:
# Encode the QCs
custom_encoded = df_combined['QC'].map(lambda x: custom_dict.get(x, {'1': 0, '2': 0, '3': 0}))
encoded = pd.DataFrame(custom_encoded.tolist(), index=df_combined.index)
encoded = encoded.add_prefix(f"{'QC'}_")  # Add column-specific prefix

df_combined = pd.concat([df_combined, encoded], axis=1)

In [ ]:
# Compute haversine distance
from scipy.spatial.distance import cdist
from math import radians, sin, cos, sqrt, atan2

def haversine_vec(latlon1, latlon2):
    # Earth radius (km)
    R = 6371  
    lat1, lon1 = np.radians(latlon1[:, 0]), np.radians(latlon1[:, 1])
    lat2, lon2 = np.radians(latlon2[:, 0]), np.radians(latlon2[:, 1])
    # Handle the shape of lat and lon to apply for different inputs
    if latlon1.shape[0] == 1 and latlon2.shape[0] == 1:
        lat1, lon1 = lat1[0], lon1[0]
        lat2, lon2 = lat2[0], lon2[0]
    # Difference of latitude in radians
    dlat = lat2 - lat1  
    dlon = lon2 - lon1
    x = np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2
    return (2 * R * atan2(sqrt(x), sqrt(1 - x))) # atan2 returns the angle


In [ ]:
# Compute spatial deviation (default value used when no matches)
def compute_spatial_dev(df, default_value=None):
    # If default value not set, assign it as monthly average
    df['expected_spatial_values'] = default_value if default_value is not None else 0
    # Centered Modis values by subtracting the monthly average
    df['Modis_c'] = df['Modis'] - df['monthly_avg']

    for day_night, group in df.groupby(['Date', 'Day_Night']):
        # If there are matches, compute spatial deviation
        if len(group) < 2:
            continue
        
        coords = group[['Latitude', 'Longitude']].values
        # Use cdist() to compute the distance efficiently
        distances = cdist(coords, coords, metric=lambda u, v: haversine_vec(u.reshape(1, -1), v.reshape(1, -1)))
        np.fill_diagonal(distances, np.inf)  # Set dist(x, x) as infinity
        weights = 1 / distances  # Inverse-distance weights
        weights /= weights.sum(axis=1, keepdims=True)
        df.loc[group.index, 'expected_spatial_values'] = weights @ group['Modis_c'].values
    
    df['spatial_dev'] = df['Modis_c'] - df['expected_spatial_values']
    return df

In [ ]:
df_combined = compute_spatial_dev(df_combined)

In [ ]:
# Compute the statistics
# Sort data by Name, Date, Day/Night
df_combined = df_combined.sort_values(['Name', 'Date', 'Day_Night'])

# Consider both day and night observtions (td1)
df_combined["Prev_Modis1"] = df_combined.groupby('Name')["Modis"].shift(1)  # Fill gaps with last available value
df_combined["Time_Diff1"] = df_combined.groupby('Name')["Date"].diff().dt.days.fillna(1) # Days since last observation

# Consider day and night observations separately (td2)
df_combined["Prev_Modis2"] = df_combined.groupby(['Name', 'Day_Night'])["Modis"].shift(1)  # Fill gaps with last available value
df_combined["Time_Diff2"] = df_combined.groupby(['Name', 'Day_Night'])["Date"].diff().dt.days.fillna(1) # Days since last observation

# Compute deviation normalized by time gap (avoid dividing by 0)
df_combined["temporal_dev1"] = (df_combined["Modis"] - df_combined["Prev_Modis1"]) / (df_combined["Time_Diff1"] + 1)
df_combined["temporal_dev2"] = (df_combined["Modis"] - df_combined["Prev_Modis2"]) / (df_combined["Time_Diff2"] + 1)

# Replace the first td1 and td2 with the first available observations
df_combined["temporal_dev1"] = df_combined.groupby('Name')['temporal_dev1'].transform(lambda x: x.fillna(x.iloc[1]))
df_combined["temporal_dev2"] = df_combined.groupby(['Name', 'Day_Night'])['temporal_dev2'].transform(lambda x: x.fillna(x.iloc[1]))

In [ ]:
df1 = df_combined.copy()

## 2. Modelling Modis (LMM)

In [ ]:
# Group for random effect
df1['Group'] = df1['Name'] + "_" + df1['Day_Night'].astype(str)

In [ ]:
# Define the mixed effects model formula (no spatial buffer)
formula = 'Modis ~ Situ + QC_1 + QC_2 + nonlinear1'

In [ ]:
# Define the mixed effects model formula (with spatial buffer)
formula = 'Modis ~ Situ + QC_1 + QC_2 + QC_3 + nonlinear1'

In [ ]:
# LMM
model = smf.mixedlm(formula, df1, groups=df1['Group'])
result = model.fit()

In [ ]:
print(result.summary())

### Check Residuals

In [ ]:
# Residual Analysis
residuals = result.resid
fitted = result.fittedvalues


res = pd.DataFrame({
    'Fitted Modis Value': fitted,
    'Residuals': residuals
})

In [ ]:
# Residual plot for LMM
plt.figure(figsize=(12, 9))
sns.scatterplot(data=res.sample(2000), x='Fitted Modis Value', y='Residuals')
plt.axhline(0, color='red', linestyle='--', linewidth=2)
plt.axhline(10, color='blue', linestyle='--', linewidth=2)
plt.axhline(-10, color='blue', linestyle='--', linewidth=2)
plt.xlabel('Fitted Values',fontsize=16)
plt.ylabel('Residuals',fontsize=16)
plt.ylim(-30,20)
plt.title("Residuals vs. Fitted Modis Values (2000 Samples, '8day + no buffer' Dataset)",fontsize=20)
plt.savefig('res_modis8_no.png')
plt.show()

In [ ]:
# QQ Plot
plt.figure(figsize=(12, 9))
qqplot(residuals, line='s')
plt.title("QQ Plot ('8day + no buffer' Dataset)", fontsize=15)
plt.ylim(-46,24)
plt.savefig('qqplot8_no.png')
plt.show()

### Fit GMM to Label Extremes

In [ ]:
# Fit GMM to residuals
gmm = GaussianMixture(n_components=2, random_state=34)
gmm.fit(res['Residuals'].values.reshape(-1, 1))

# Get probabilities for each component
probs = gmm.predict_proba(res['Residuals'].values.reshape(-1, 1))

# Check the center of each component
print(gmm.means_)

# Probability of being an extreme (lower mean)
res['Extreme_Prob'] = probs[:, np.argmin(gmm.means_)]
df1['Extreme_Prob'] = res['Extreme_Prob']

In [ ]:
# Histogram of probability of errors
plt.figure(figsize=(12, 9))
sns.histplot(res['Extreme_Prob'], bins=18, kde=True)
plt.xlabel('Extreme_Prob', fontsize=16)
plt.axvline(0.9, color='red', linestyle='--', linewidth=2)
plt.yscale('log')
plt.title("Histogram of Probability of Extremes (Log-Scaled, '8day + no buffer' Dataset)", fontsize=18)
plt.savefig('his_ex8_no.png')
plt.show()

In [ ]:
# Set label for large Extreme_Prob
df1['label'] = (df1['Extreme_Prob'] > 0.9)
df1['label'] = df1['label'].astype(int)

## 3. Make Prediction of Label (Logistic Regression)

In [ ]:
# Define the label model formula (no spatial buffer)
formula_l = 'label ~ Modis + QC_1:Day_Night + QC_2 + nonlinear2 + temporal_dev1 + temporal_dev2 + temporal_dev3:Day_Night + spatial_dev + Modis:day_of_year_sin + Modis:day_of_year_cos'

In [ ]:
# Define the formula (no spatial buffer) after removing insignificant variables
formula_l = 'label ~ Modis + QC_1:Day_Night + QC_2 + temporal_dev1 + temporal_dev2 + temporal_dev3:Day_Night + spatial_dev + Modis:day_of_year_sin + Modis:day_of_year_cos'

In [ ]:
# Define the label model formula (with spatial buffer)
formula_l = 'label ~ Modis + QC_1:Day_Night + QC_2 + QC_3 + nonlinear2 + temporal_dev1 + temporal_dev2 + temporal_dev3:Day_Night + spatial_dev + Modis:day_of_year_sin + Modis:day_of_year_cos'

In [ ]:
# Define the label model formula (with spatial buffer) after removing insignificant variables
formula_l = 'label ~ Modis + QC_1:Day_Night + QC_2 + QC_3 + temporal_dev1 + temporal_dev2 + temporal_dev3:Day_Night + spatial_dev + Modis:day_of_year_sin + Modis:day_of_year_cos'

In [ ]:
# Define the label model formula (with spatial buffer) after removing insignificant variables
formula_l = 'label ~ Modis + QC_2 + QC_3 + temporal_dev1 + temporal_dev2 + temporal_dev3:Day_Night + spatial_dev + Modis:day_of_year_sin + Modis:day_of_year_cos'

In [ ]:
model_l = smf.logit(formula_l, df1)
result_l = model_l.fit(cov_type='HC0')

In [ ]:
print(result_l.summary())

In [ ]:
# Check prediction
prediction_l = pd.DataFrame({'prob':result_l.predict(),
                            'label':df1['label'].values,
                            'Extreme_Prob':df1['Extreme_Prob'].values})
df1['fit_prob'] = result_l.predict()
prediction_l[prediction_l['Extreme_Prob']>0.5].sample(20)

In [ ]:
# Statistics and TPR/FPR
print(len(df1[df1['Extreme_Prob']>0.9]))
print(len(df1[df1['Extreme_Prob']>0.9])/len(df1))
print(len(df1[df1['fit_prob']>0.5])/len(df1))
print(len(df1[(df1['fit_prob']>0.5)&(df1['Extreme_Prob']>0.9)])/len(df1[df1['Extreme_Prob']>0.9]))
print(len(df1[(df1['fit_prob']>0.5)&(df1['Extreme_Prob']<=0.9)])/len(df1[df1['Extreme_Prob']<=0.9]))

In [ ]:
# Prediction plot
plt.figure(figsize=(12, 9))
sns.scatterplot(data=prediction_l.sample(2000), x='Extreme_Prob', y='prob', hue='label')
plt.axhline(0.5, color='red', linestyle='--', linewidth=2)
plt.xlabel('Extreme_Prob',fontsize=16)
plt.ylabel('Predicted Probability',fontsize=16)
plt.legend(fontsize=14)
plt.ylim(-0.02,1.02)
plt.title("Fitted Probability vs. Extreme_Prob (2000 Samples from '8day + no buffer' dataset)",fontsize=18)
plt.savefig('label_prob8_no.png')
plt.show()

## 3. Fit Prediction Model (Use rpy2 to Implement GAMM in R)

In [ ]:
# Remove fitted (probability >= 0.5)
df2 = df1[df1['fit_prob'] < 0.5]

In [ ]:
# Encode Day_Night and QC_1 into one multi-class variable
df2['D_QC_1'] = 'D' + df2['Day_Night'].astype(str) + 'Q' + df2['QC_1'].astype(str)

In [ ]:
import rpy2.robjects as ro
from rpy2.robjects import r, pandas2ri
from rpy2.robjects.packages import importr

# Enable automatic conversion between Python and R DataFrames
pandas2ri.activate()

In [ ]:
# Load R libraries
ro.r('install.packages("mgcv", repos="http://cran.rstudio.com")')
ro.r('install.packages("ggplot2", repos="http://cran.rstudio.com")')
mgcv = importr('mgcv')
ggplot2 = importr('ggplot2')
ro.r('install.packages("caret", repos="http://cran.rstudio.com")')
caret = importr('caret')
base = importr('base')

In [ ]:
pd.DataFrame.iteritems = pd.DataFrame.items  # Avoid incompatibility
r_df2 = pandas2ri.py2rpy(df2)

In [ ]:
# Define R function to train GAMM (final model)
ro.r('''
    fit_gam <- function(df, save_path="model.rds") {
        df$QC_1 <- as.factor(df$QC_1)
        df$QC_2 <- as.factor(df$QC_2)
        df$QC_3 <- as.factor(df$QC_3)
        df$Day_Night <- as.factor(df$Day_Night)
        df$D_QC_1 <- as.factor(df$D_QC_1)
        df$Group <- as.factor(df$Group)

        preds <- data.frame()

        # Fit GAMM
        model <- gam(Situ ~ Modis + s(Modis, by=D_QC_1, k=5) + QC_1 + QC_2 + QC_3 + spatial_dev + s(Group, bs="re"),
                     data=df, method="REML", select=TRUE)

        print(summary(model))

        # Prediction
        df$pred <- predict(model, newdata=df)
        df$residual <- df$Situ - df$pred

        spline_values <- predict(model, newdata=df, type="terms")
        spline_df <- as.data.frame(spline_values)
        # Rename splines
        colnames(spline_df) <- paste0("Spline_", colnames(spline_df))

        df <- cbind(df, spline_df)
        

        mae_value <- mean(abs(df$residual))


        # Coefficients
        coefs <- coef(model)
        coef_names <- names(coefs)

        # Identify types of coefficients
        fixed_coefs <- coefs[!grepl("s\\\\(", coef_names)]
        spline_coefs <- coefs[grepl("s\\\\(", coef_names)]
        random_coefs <- coefs[grepl("s\\\\(Group", coef_names)]

        fixed_effects <- data.frame(Term=names(fixed_coefs), Coefficient=fixed_coefs)
        spline_effects <- data.frame(Term=names(spline_coefs), Coefficient=spline_coefs)
        random_effects <- data.frame(Term=names(random_coefs), Coefficient=random_coefs)
        
        saveRDS(model, save_path)

        return(list(df_pred=df,
                    fixed_effects=fixed_effects,
                    spline_effects=spline_effects, 
                    random_effects=random_effects,
                    mae = mae_value))
    }
''')

In [ ]:
# Call the function from Python
fit_gam = ro.globalenv['fit_gam']
df_pred_cv = fit_gam(r_df2)

In [ ]:
# R code for GAMM model with 5-fold cv
ro.r('''
    cross_validate_gam <- function(df, k_folds=5) {
        set.seed(34)
        # Encode binary and group variables as factors
        df$QC_1 <- as.factor(df$QC_1)
        df$QC_2 <- as.factor(df$QC_2)
        df$QC_3 <- as.factor(df$QC_3)
        df$Day_Night <- as.factor(df$Day_Night)
        df$D_QC_1 <- as.factor(df$D_QC_1)
        df$Group <- as.factor(df$Group)

        folds <- createFolds(df$Situ, k = k_folds, list = TRUE)
        all_preds <- data.frame()
        mae_values <- numeric(k_folds)

        # Dataframes to store coefficients and predictions for each fold
        fixed_effects_all <- data.frame()
        spline_effects_all <- data.frame()
        categorical_effects_all <- data.frame()
        random_effects_all <- data.frame()

        for (i in seq_along(folds)) {
            test_idx <- folds[[i]]
            train_df <- df[-test_idx, ]
            test_df <- df[test_idx, ]

            model <- gam(Situ ~ Modis + s(Modis, by=D_QC_1, k=5) + QC_1 + QC_2 + QC_3 + spatial_dev + s(Group, bs="re"),
                         data=train_df, method="REML", select=TRUE)
            
            print(summary(model))

            # Prediction on test sets
            test_df$pred <- predict(model, newdata=test_df)
            test_df$residual <- test_df$Situ - test_df$pred

            spline_values <- predict(model, newdata=test_df, type="terms")
            spline_df <- as.data.frame(spline_values)
            # Rename splines
            colnames(spline_df) <- paste0("Spline_", colnames(spline_df))

            test_df <- cbind(test_df, spline_df)
            # Keep track of the fold for comparison
            test_df$fold <- i

            mae_values[i] <- mean(abs(test_df$residual))

            # Store predictions
            all_preds <- rbind(all_preds, test_df)

            # Coefficients
            coefs <- coef(model)
            coef_names <- names(coefs)

            # Identify types of coefficients
            fixed_coefs <- coefs[!grepl("s\\\\(", coef_names)]
            spline_coefs <- coefs[grepl("s\\\\(", coef_names)]
            random_coefs <- coefs[grepl("s\\\\(Group", coef_names)]

            fixed_effects_all <- rbind(fixed_effects_all, 
                                       data.frame(Fold=i, Term=names(fixed_coefs), Coefficient=fixed_coefs))
            spline_effects_all <- rbind(spline_effects_all, 
                                        data.frame(Fold=i, Term=names(spline_coefs), Coefficient=spline_coefs))
            random_effects_all <- rbind(random_effects_all, 
                                        data.frame(Fold=i, Term=names(random_coefs), Coefficient=random_coefs))
        }
    
        return(list(df_pred=all_preds,
                    fixed_effects=fixed_effects_all,
                    spline_effects=spline_effects_all, 
                    random_effects=random_effects_all,
                    mae = mean(mae_values)))
    }
''')

In [ ]:
# Run cross-validation and get predictions, coefficients and MAE
cross_validate_gam = ro.globalenv['cross_validate_gam']
df_pred_cv = cross_validate_gam(r_df2)

In [ ]:
final_pred = df_pred_cv[0]
fixed_coef = df_pred_cv[1]
splines = df_pred_cv[2]
group_coef = df_pred_cv[3]
final_mae = df_pred_cv[4][0]
print(f"MAE of GAM: {final_mae:.4f}")

In [ ]:
monthly_pred = final_pred.groupby(['Name', 'Year', 'Month', 'Day_Night'])[['Situ', 'pred']].mean().reset_index().dropna(subset=['Situ','pred'], how='all')
monthly_mae = mean_absolute_error(monthly_pred['Situ'], monthly_pred['pred'])
print(f"Monthly MAE of GAM: {monthly_mae:.4f}")

In [ ]:
print(len(final_pred))
print((final_pred['residual'].abs()>5).sum()/len(final_pred))

In [ ]:
# Save results
final_pred.to_parquet("1day_no_pred.parquet", index=False)
fixed_coef.to_parquet("1day_no_fixed.parquet", index=False)
splines.to_parquet("1day_no_splines.parquet", index=False)
group_coef.to_parquet("1day_no_group.parquet", index=False)

### Comparison with no splines

In [ ]:
# R code for GAMM model with 5-fold cv (no splines)
ro.r('''
    cross_validate_gam <- function(df, k_folds=5) {
        set.seed(34)
        # Encode binary and group variables as factors
        df$QC_1 <- as.factor(df$QC_1)
        df$QC_2 <- as.factor(df$QC_2)
        df$QC_3 <- as.factor(df$QC_3)
        df$Day_Night <- as.factor(df$Day_Night)
        df$D_QC_1 <- as.factor(df$D_QC_1)
        df$Group <- as.factor(df$Group)

        folds <- createFolds(df$Situ, k = k_folds, list = TRUE)
        all_preds <- data.frame()
        mae_values <- numeric(k_folds)

        # Dataframes to store coefficients and predictions for each fold
        fixed_effects_all <- data.frame()
        categorical_effects_all <- data.frame()
        random_effects_all <- data.frame()

        for (i in seq_along(folds)) {
            test_idx <- folds[[i]]
            train_df <- df[-test_idx, ]
            test_df <- df[test_idx, ]

            model <- gam(Situ ~ Modis + QC_1 + QC_2 + QC_3 + spatial_dev + s(Group, bs="re"),
                         data=train_df, method="REML", select=TRUE)
            
            print(summary(model))

            # Prediction on test sets
            test_df$pred <- predict(model, newdata=test_df)
            test_df$residual <- test_df$Situ - test_df$pred

            spline_values <- predict(model, newdata=test_df, type="terms")
            spline_df <- as.data.frame(spline_values)
            # Rename splines
            colnames(spline_df) <- paste0("Spline_", colnames(spline_df))

            test_df <- cbind(test_df, spline_df)
            # Keep track of the fold for comparison
            test_df$fold <- i

            mae_values[i] <- mean(abs(test_df$residual))

            # Store predictions
            all_preds <- rbind(all_preds, test_df)

            # Coefficients
            coefs <- coef(model)
            coef_names <- names(coefs)

            # Identify types of coefficients
            fixed_coefs <- coefs[!grepl("s\\\\(", coef_names)]
            random_coefs <- coefs[grepl("s\\\\(Group", coef_names)]

            fixed_effects_all <- rbind(fixed_effects_all, 
                                       data.frame(Fold=i, Term=names(fixed_coefs), Coefficient=fixed_coefs))
            random_effects_all <- rbind(random_effects_all, 
                                        data.frame(Fold=i, Term=names(random_coefs), Coefficient=random_coefs))
        }
    
        return(list(df_pred=all_preds,
                    fixed_effects=fixed_effects_all,
                    random_effects=random_effects_all,
                    mae = mean(mae_values)))
    }
''')

In [ ]:
# Run cross-validation and get predictions, coefficients and MAE
cross_validate_gam = ro.globalenv['cross_validate_gam']
df_pred_cv = cross_validate_gam(r_df2)

In [ ]:
final_pred = df_pred_cv[0]
fixed_coef = df_pred_cv[1]
group_coef = df_pred_cv[2]
final_mae = df_pred_cv[3][0]
print(f"MAE of GAM: {final_mae:.4f}")

In [ ]:
monthly_pred = final_pred.groupby(['Name', 'Year', 'Month', 'Day_Night'])[['Situ', 'pred']].mean().reset_index().dropna(subset=['Situ','pred'], how='all')
monthly_mae = mean_absolute_error(monthly_pred['Situ'], monthly_pred['pred'])
print(f"Monthly MAE of GAM: {monthly_mae:.4f}")

In [ ]:
# Load saved results
final_pred = pd.read_parquet('/kaggle/input/result-parquet/8day_no_pred.parquet')

In [ ]:
print(len(final_pred))
print((final_pred['residual'].abs()>5).sum()/len(final_pred))

In [ ]:
monthly_pred = final_pred.groupby(['Name', 'Year', 'Month', 'Day_Night'])[['Situ', 'pred']].mean().reset_index().dropna(subset=['Situ','pred'], how='all')

In [ ]:
print(len(monthly_pred))
print(((monthly_pred['pred']-monthly_pred['Situ']).abs()>3).sum()/len(monthly_pred))

In [ ]:
final_pred.info()

In [ ]:
# Plot splines from different folds
plt.figure(figsize=(12, 9))
sns.lineplot(data=final_pred[final_pred['D_QC_1']=='D0Q1'], x='Modis', y='Spline_s(Modis):D_QC_1D0Q1', hue='fold', palette="tab10")
plt.title("Splines for QC_1=1 at Night Across 5 Folds ('8day + no buffer' Dataset)", fontsize=20)
#plt.axhline(0, color='red', linestyle='--', linewidth=1)
plt.xlabel('Modis', fontsize=16)
plt.ylabel('Spline_s(Modis):D_QC_1D0Q1', fontsize=16)
#plt.ylim(-10,7)
plt.savefig('sp_d0q1_8day_no.png')
plt.show()

In [ ]:
# Plot residuals vs. fitted values
plt.figure(figsize=(12, 9))
sns.scatterplot(data=final_pred.sample(2000), x='pred', y='residual')
plt.axhline(0, color='red', linestyle='--', linewidth=2)
plt.axhline(5, color='blue', linestyle='--', linewidth=1.5)
plt.axhline(-5, color='blue', linestyle='--', linewidth=1.5)
plt.xlabel('Fitted Situ Values',fontsize=16)
plt.ylabel('Residuals',fontsize=16)
plt.ylim(-25,25)
plt.title("Residual Plot (2000 Samples for 'Daily + 1km' Dataset)",fontsize=20)
plt.savefig('res_final_1.png')
plt.show()

In [ ]:
# QQ Plot
plt.figure(figsize=(12, 9))
qqplot(final_pred['residual'], line='r')
plt.title("QQ Plot (Final Model) for '8day + 1km' Dataset", fontsize=15)
plt.ylim(-17,27)
plt.savefig('qqplot_final_8.png')
plt.show()

### Load Saved Models

In [ ]:
from rpy2.robjects.conversion import localconverter

# Load saved models
def load_rds(rds_path, df, df0):
    model = base.readRDS(rds_path)

    # Predictions
    pred = r['predict'](model, newdata=df)
    
    # Spline terms
    spline_matrix = r['predict'](model, newdata=df, type="terms", se_fit=True)
    #spline_colnames = list(ro.r.colnames(spline_matrix))

    # Convert to Python dataframes
    with localconverter(robjects.default_converter + pandas2ri.converter):
        pred_df = robjects.conversion.rpy2py(pred)
        spline_df = robjects.conversion.rpy2py(spline_matrix)
        data_df = robjects.conversion.rpy2py(df)

    # Combine spline terms and prediction
    results_df = df0.copy()
    results_df["pred"] = pred_df
    results_df = pd.concat([results_df, pd.DataFrame(spline_df)], axis=1)

    # Extract coefficients
    coefs = model.rx2("coefficients")
    coefs_dict = {name: coefs.rx2(name)[0] for name in coefs.names}
    coefs_df = pd.DataFrame(list(coefs_dict.items()), columns=["term", "coefficient"])

    return results_df, coefs_df


In [ ]:
# Define R function to train GAMM (from saved models)
ro.r('''
    extract_rds <- function(rds_path, df) {
        preds <- data.frame()

        model <- readRDS(rds_path)

        print(summary(model))

        # Prediction
        df$pred <- predict(model, newdata=df)
        df$residual <- df$Situ - df$pred

        spline_values <- predict(model, newdata=df, type="terms")
        spline_df <- as.data.frame(spline_values)
        # Rename splines
        colnames(spline_df) <- paste0("Spline_", colnames(spline_df))

        df <- cbind(df, spline_df)
        

        mae_value <- mean(abs(df$residual))


        # Coefficients
        coefs <- coef(model)
        coef_names <- names(coefs)

        # Identify types of coefficients
        fixed_coefs <- coefs[!grepl("s\\\\(", coef_names)]
        spline_coefs <- coefs[grepl("s\\\\(", coef_names)]
        random_coefs <- coefs[grepl("s\\\\(Group", coef_names)]

        fixed_effects <- data.frame(Term=names(fixed_coefs), Coefficient=fixed_coefs)
        spline_effects <- data.frame(Term=names(spline_coefs), Coefficient=spline_coefs)
        random_effects <- data.frame(Term=names(random_coefs), Coefficient=random_coefs)

        return(list(df_pred=df,
                    fixed_effects=fixed_effects,
                    spline_effects=spline_effects, 
                    random_effects=random_effects,
                    mae = mae_value))
    }
''')

In [ ]:
extract_rds = ro.globalenv['extract_rds']
all_results = extract_rds('/kaggle/input/result-parquet/model_1day.rds', r_df2)

In [ ]:
final_pred = all_results[0]
fixed_coef = all_results[1]
splines = all_results[2]
group_coef = all_results[3]
final_mae = all_results[4][0]
print(f"MAE of GAM: {final_mae:.4f}")

In [ ]:
final_pred.info()

In [ ]:
final_pred['Spline_Modis_D0Q0'] = fixed_coef['Coefficient'][1]*final_pred['Modis'] + final_pred['Spline_s(Modis):D_QC_1D0Q0'] + fixed_coef['Coefficient'][0]
final_pred['Spline_Modis_D0Q1'] = fixed_coef['Coefficient'][1]*final_pred['Modis'] + final_pred['Spline_s(Modis):D_QC_1D0Q1'] + fixed_coef['Coefficient'][0]
final_pred['Spline_Modis_D1Q0'] = fixed_coef['Coefficient'][1]*final_pred['Modis'] + final_pred['Spline_s(Modis):D_QC_1D1Q0'] + fixed_coef['Coefficient'][0]
final_pred['Spline_Modis_D1Q1'] = fixed_coef['Coefficient'][1]*final_pred['Modis'] + final_pred['Spline_s(Modis):D_QC_1D1Q1'] + fixed_coef['Coefficient'][0]

In [ ]:
# Map spline terms for each category
dq_map = {
    'D0Q0': 'Spline_Modis_D0Q0',
    'D0Q1': 'Spline_Modis_D0Q1',
    'D1Q0': 'Spline_Modis_D1Q0',
    'D1Q1': 'Spline_Modis_D1Q1'
}

dq_diff_map = {
    'D0Q0': 'Spline_Diff_D0Q0',
    'D0Q1': 'Spline_Diff_D0Q1',
    'D1Q0': 'Spline_Diff_D1Q0',
    'D1Q1': 'Spline_Diff_D1Q1'
}

style_map = {
    'D0Q0': {'color': 'blue', 'linewidth': 2},
    'D0Q1': {'color': 'green', 'linewidth': 2},
    'D1Q0': {'color': 'red', 'linewidth': 2},
    'D1Q1': {'color': 'orange', 'linewidth': 2}
}

In [ ]:
# Plot splines from different QC_1 and Day_Night
plt.figure(figsize=(12, 9))
for dq_val, sp_col in dq_map.items():
    sub_df = final_pred[final_pred['D_QC_1'] == dq_val]
    sns.lineplot(
        data=sub_df,
        x='Modis',
        y=sp_col,
        label=f'{sp_col}',
        **style_map[dq_val]
    )

# Add auxiliary lines
x = final_pred['Modis'].sort_values().unique()
plt.plot(x, x, color='black', linestyle='--', label='y = x')
plt.plot(x, x + fixed_coef['Coefficient'][0], color='gray', linestyle='--', label='y = x + Intercept')
#plt.plot(x[x<0], x[x<0] + fixed_coef['Coefficient'][0] + 3, color='purple', linestyle='--', label='y = x + Intercept + 3')
plt.legend(fontsize=12)
plt.title("All Splines Terms for Modis ('Daily + 1km' Dataset)", fontsize=20)
plt.xlabel('Modis', fontsize=16)
plt.ylabel('Spline Terms for Modis', fontsize=16)
#plt.ylim(-12,5)
plt.savefig('modis_sp_all_1.png')
plt.show()

In [ ]:
final_pred['Spline_Diff_D0Q0'] = final_pred['Spline_Modis_D0Q0'] - final_pred['Modis']
final_pred['Spline_Diff_D0Q1'] = final_pred['Spline_Modis_D0Q1'] - final_pred['Modis']
final_pred['Spline_Diff_D1Q0'] = final_pred['Spline_Modis_D1Q0'] - final_pred['Modis']
final_pred['Spline_Diff_D1Q1'] = final_pred['Spline_Modis_D1Q1'] - final_pred['Modis']

In [ ]:
pred_sel = final_pred[final_pred['Modis']>5]

In [ ]:
# Plot splines from different QC_1 and Day_Night
plt.figure(figsize=(12, 9))
for dq_val, sp_col in dq_diff_map.items():
    sub_df = final_pred[final_pred['D_QC_1'] == dq_val]
    sns.lineplot(
        data=sub_df,
        x='Modis',
        y=sp_col,
        label=f'{sp_col}',
        **style_map[dq_val]
    )

# Add auxiliary lines
x = final_pred['Modis'].sort_values().unique()
plt.axhline(0, color='black', linestyle='--', linewidth=2)
plt.axhline(fixed_coef['Coefficient'][0], color='gray', linestyle='--', linewidth=2)
plt.legend(fontsize=12)
plt.title("Difference Between Splines Terms and Modis ('8-Day + 1km' Dataset)", fontsize=20)
plt.xlabel('Modis', fontsize=16)
plt.ylabel('Spline Terms for Modis', fontsize=16)
plt.ylim(-7.8,8.2)
plt.savefig('modis_sp_all_1d.png')
plt.show()

### Check Random Effects

In [ ]:
final_pred = pd.read_parquet('/kaggle/input/result-parquet/8day_1km_pred.parquet')

In [ ]:
group_effects = final_pred[['Group', 'Spline_s(Group)']].drop_duplicates()

In [ ]:
group_effects.sort_values(by='Spline_s(Group)', ascending=False).head(10)

In [ ]:
# Histogram of random effects
plt.figure(figsize=(12, 9))
sns.histplot(group_effects['Spline_s(Group)'], bins=16, kde=True)
plt.xlabel('Random Effects', fontsize=16)
plt.title('Histogram of Random Effects for Groups', fontsize=20)
plt.savefig('his_rad_8.png')
plt.show()

In [ ]:
# Comparison with no random effects
final_pred['pred_no_ra'] = final_pred['pred'] - final_pred['Spline_s(Group)']
mae_no_ra = mean_absolute_error(final_pred['Situ'], final_pred['pred_no_ra'])
monthly_pred_no_ra = final_pred.groupby(['Name', 'Year', 'Month', 'Day_Night'])[['Situ', 'pred_no_ra']].mean().reset_index().dropna(subset=['Situ','pred_no_ra'], how='all')
monthly_mae_no_ra = mean_absolute_error(monthly_pred_no_ra['Situ'], monthly_pred_no_ra['pred_no_ra'])
print(f"MAE of GAM: {mae_no_ra:.4f}")
print(f"Monthly MAE of GAM: {monthly_mae_no_ra:.4f}")
print(((final_pred['pred_no_ra']-final_pred['Situ']).abs()>5).sum()/len(final_pred))
print(((monthly_pred_no_ra['pred_no_ra']-monthly_pred_no_ra['Situ']).abs()>3).sum()/len(monthly_pred_no_ra))